In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os 
import glob
import re
import toolz.curried as tl
import datetime

In [2]:
DATA_FILES = glob.glob(os.path.join(os.getcwd(), "data", "original_data", "*.xls*"))

In [3]:
print(*map(os.path.basename, DATA_FILES))

Houston_Complete_Sorted.xlsm Limestone_Complete_Sorted.xlsm Madison_Irrigated_2.xls Madison_Rainfed_2.xls


Quickly looking through the data in excel we can see that madison is separated into two files for rainfed and irrigated where as houston and limestone have a column for a flag to tell if it is rainfed or irrigated. First step is to get all the files to have similar formats so I will aggregate the two madison files into one.

In [4]:
def find_file(regex, files):
    for file in files:
        if re.search(regex, file):
            return file
    raise ValueError("No File Found.")

In [5]:
madison_rf_df = pd.read_excel(find_file(r".*Madison_Rain.*", DATA_FILES))
limestone_df = pd.read_excel(find_file(r".*Limestone.*", DATA_FILES))
houston_df = pd.read_excel(find_file(r".*Houston.*", DATA_FILES))
madison_irr_df = pd.read_excel(find_file(r".*Madison_Irr.*", DATA_FILES))

In [6]:
rf_cols = sorted(madison_rf_df.columns)
irr_cols = sorted(madison_irr_df.columns)
for i in range(len(rf_cols)):
    if rf_cols[i] != irr_cols[i]:
        print(f"index {i}\n{rf_cols[i]} != {irr_cols[i]}")

In [7]:
madison_rf_df["rf_irr"] = 0
madison_irr_df["rf_irr"] = 1

In [8]:
madison_rf_df.head()

,Longitude,Latitude,MOD13Q1_006__250m_16_days_EVI_doy2011081_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2012081_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2011097_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2011113_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2011129_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2011145_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2011161_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2011177_aid0001_tif,...,MOD13Q1_006__250m_16_days_EVI_doy2012113_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2012129_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2012145_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2012161_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2012177_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2012193_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2012209_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2012225_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2012241_aid0001_tif,rf_irr
0,-86.779999,34.990601,3413,3747,4421,3276,2311,2988,2734,4211,...,3416,2782,4794,5903,4964,5358.0,5287.0,3766,3680.0,0
1,-86.777496,34.990601,3281,3747,4421,3276,2445,3072,3030,4211,...,3400,3202,5065,4745,4172,4764.0,4815.0,3797,3916.0,0
2,-86.764801,34.990601,4045,4696,4981,4962,3469,4083,3838,5155,...,4230,3126,4240,5217,3612,5986.0,5905.0,4957,3974.0,0
3,-86.757202,34.990601,4167,5114,5000,3959,4367,3958,3031,3781,...,4808,4320,5187,4801,4535,5090.0,4958.0,4200,3011.0,0
4,-86.754600,34.990601,4256,5114,5000,3822,3801,2854,2302,3863,...,3525,3904,5480,5388,4324,4456.0,4866.0,3261,3443.0,0


In [9]:
madison_df = pd.concat([madison_rf_df, madison_irr_df])
madison_df["county"] = "Madison"

C:\Users\paul.abers\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [10]:
madison_rf_df.info()
madison_irr_df.info()
madison_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5053 entries, 0 to 5052
Data columns (total 25 columns):
Longitude                                               5053 non-null float64
Latitude                                                5053 non-null float64
MOD13Q1_006__250m_16_days_EVI_doy2011081_aid0001_tif    5053 non-null int64
MOD13Q1_006__250m_16_days_EVI_doy2012081_aid0001_tif    5053 non-null int64
MOD13Q1_006__250m_16_days_EVI_doy2011097_aid0001_tif    5053 non-null int64
MOD13Q1_006__250m_16_days_EVI_doy2011113_aid0001_tif    5053 non-null int64
MOD13Q1_006__250m_16_days_EVI_doy2011129_aid0001_tif    5053 non-null int64
MOD13Q1_006__250m_16_days_EVI_doy2011145_aid0001_tif    5053 non-null int64
MOD13Q1_006__250m_16_days_EVI_doy2011161_aid0001_tif    5053 non-null int64
MOD13Q1_006__250m_16_days_EVI_doy2011177_aid0001_tif    5053 non-null int64
MOD13Q1_006__250m_16_days_EVI_doy2011193_aid0001_tif    5051 non-null float64
MOD13Q1_006__250m_16_days_EVI_doy2011209_aid0001_ti

In [11]:
madison_df.head()

,Latitude,Longitude,MOD13Q1_006__250m_16_days_EVI_doy2011081_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2011097_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2011113_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2011129_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2011145_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2011161_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2011177_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2011193_aid0001_tif,...,MOD13Q1_006__250m_16_days_EVI_doy2012129_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2012145_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2012161_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2012177_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2012193_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2012209_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2012225_aid0001_tif,MOD13Q1_006__250m_16_days_EVI_doy2012241_aid0001_tif,rf_irr,county
0,34.990601,-86.779999,3413,4421,3276,2311,2988,2734,4211,5231.0,...,2782,4794,5903,4964,5358.0,5287.0,3766,3680.0,0,Madison
1,34.990601,-86.777496,3281,4421,3276,2445,3072,3030,4211,5960.0,...,3202,5065,4745,4172,4764.0,4815.0,3797,3916.0,0,Madison
2,34.990601,-86.764801,4045,4981,4962,3469,4083,3838,5155,5949.0,...,3126,4240,5217,3612,5986.0,5905.0,4957,3974.0,0,Madison
3,34.990601,-86.757202,4167,5000,3959,4367,3958,3031,3781,5712.0,...,4320,5187,4801,4535,5090.0,4958.0,4200,3011.0,0,Madison
4,34.990601,-86.754600,4256,5000,3822,3801,2854,2302,3863,5656.0,...,3904,5480,5388,4324,4456.0,4866.0,3261,3443.0,0,Madison


In [12]:
houston_df.head()

,X,Y,Unnamed: 2,Unnamed: 3,RF_Irr,EVI_2011_Apr_7_Houston_tif_rf_1,EVI_2011_Apr_23_Houston_tif_rf,EVI_2011_May_9_Houston_tif_rf,EVI_2011_May_25_Houston_tif_rf,EVI_2011_Jun_10_Houston_tif_rf,...,EVI_2012_Apr_6_Houston_tif_rf,EVI_2012_Apr_22_Houston_tif_rf,EVI_2012_May_8_Houston_tif_rf,EVI_2012_May_24_Houston_tif_rf,EVI_2012_Jun_9_Houston_tif_rf,EVI_2012_Jun_25_Houston_tif_rf,EVI_2012_Jul_11_Houston_tif_rf,EVI_2012_Jul_27_Houston_tif_rf,EVI_2012_Aug_12_Houston_tif_rf,EVI_2012_Aug_28_Houston_tif_rf
0,651082,3.46556e+06,31.314623,-85.412285,0.0,2800,3114,1873,2989,3299,...,4270,3417,2958,2656,4659,3069,5409,5876,5800,5738
1,653630,3.46556e+06,31.314289,-85.385513,0.0,1530,1453,1416,1364,1703,...,1699,1418,1523,1683,2726,2976,4179,4016,5509,4558
2,653862,3.46556e+06,31.314259,-85.383079,0.0,2076,1500,1602,2063,1703,...,1435,1754,1960,1926,2726,2976,4364,4848,4623,5547
3,651082,3.46533e+06,31.312532,-85.412315,0.0,2029,1400,1877,1343,1788,...,3220,1678,2599,2452,2149,2710,3564,5173,6491,6901
4,652009,3.46533e+06,31.312412,-85.402580,0.0,1854,2119,1611,1549,2393,...,1547,1861,1993,2278,4305,4462,4609,5122,5654,5367


In [13]:
houston_df = houston_df.rename(columns = {"RF_Irr":"rf_irr", "Unnamed: 2":"Latitude", "Unnamed: 3": "Longitude"})
houston_df["county"] = "Houston"

In [14]:
houston_df.head()

,X,Y,Latitude,Longitude,rf_irr,EVI_2011_Apr_7_Houston_tif_rf_1,EVI_2011_Apr_23_Houston_tif_rf,EVI_2011_May_9_Houston_tif_rf,EVI_2011_May_25_Houston_tif_rf,EVI_2011_Jun_10_Houston_tif_rf,...,EVI_2012_Apr_22_Houston_tif_rf,EVI_2012_May_8_Houston_tif_rf,EVI_2012_May_24_Houston_tif_rf,EVI_2012_Jun_9_Houston_tif_rf,EVI_2012_Jun_25_Houston_tif_rf,EVI_2012_Jul_11_Houston_tif_rf,EVI_2012_Jul_27_Houston_tif_rf,EVI_2012_Aug_12_Houston_tif_rf,EVI_2012_Aug_28_Houston_tif_rf,county
0,651082,3.46556e+06,31.314623,-85.412285,0.0,2800,3114,1873,2989,3299,...,3417,2958,2656,4659,3069,5409,5876,5800,5738,Houston
1,653630,3.46556e+06,31.314289,-85.385513,0.0,1530,1453,1416,1364,1703,...,1418,1523,1683,2726,2976,4179,4016,5509,4558,Houston
2,653862,3.46556e+06,31.314259,-85.383079,0.0,2076,1500,1602,2063,1703,...,1754,1960,1926,2726,2976,4364,4848,4623,5547,Houston
3,651082,3.46533e+06,31.312532,-85.412315,0.0,2029,1400,1877,1343,1788,...,1678,2599,2452,2149,2710,3564,5173,6491,6901,Houston
4,652009,3.46533e+06,31.312412,-85.402580,0.0,1854,2119,1611,1549,2393,...,1861,1993,2278,4305,4462,4609,5122,5654,5367,Houston


In [15]:
limestone_df.head()

,X,Y,Latitude,Longitude,Irr_Rf,EVI_2011_APR_7_1,EVI_2011_APR_23_,EVI_2011_MAY_9_L,EVI_2011_MAY_25_,EVI_2011_JUN_10_,...,EVI_2012_APR_6_L,EVI_2012_APR_22_,EVI_2012_MAY_8_L,EVI_2012_MAY_24_,EVI_2012_JUN_9_L,EVI_2012_JUN_25_,EVI_2012_JUL_11_,EVI_2012_JUL_27_,EVI_2012_AUG_12_,EVI_2012_AUG_28_
0,481294,3.86912e+06,34.964485,-87.204903,0.0,3703,4247,4079,4472,4666,...,5079,4006,2625,3389,3895,3660,6504,7270,6466,6445
1,497047,3.86912e+06,34.964657,-87.032349,0.0,3256,2695,2804,5184,3607,...,2830,3187,3997,4780,4882,3973,4344,4928,3651,6758
2,497510,3.86912e+06,34.964657,-87.027275,0.0,2983,3032,5763,2620,4831,...,2643,3491,3357,4701,4137,3710,4007,4268,4408,2721
3,480831,3.86889e+06,34.962391,-87.209976,0.0,3253,4592,4797,5969,3677,...,3875,4541,3599,4573,4612,4019,5942,5880,6815,5641
4,497278,3.86889e+06,34.962566,-87.029808,0.0,3483,3010,4030,2521,2799,...,3308,3166,3811,5268,4506,3898,6482,4144,3887,2598


In [16]:
limestone_df = limestone_df.rename(columns={"Irr_Rf":"rf_irr"})
limestone_df["county"] = "Limestone"
limestone_df.head()

,X,Y,Latitude,Longitude,rf_irr,EVI_2011_APR_7_1,EVI_2011_APR_23_,EVI_2011_MAY_9_L,EVI_2011_MAY_25_,EVI_2011_JUN_10_,...,EVI_2012_APR_22_,EVI_2012_MAY_8_L,EVI_2012_MAY_24_,EVI_2012_JUN_9_L,EVI_2012_JUN_25_,EVI_2012_JUL_11_,EVI_2012_JUL_27_,EVI_2012_AUG_12_,EVI_2012_AUG_28_,county
0,481294,3.86912e+06,34.964485,-87.204903,0.0,3703,4247,4079,4472,4666,...,4006,2625,3389,3895,3660,6504,7270,6466,6445,Limestone
1,497047,3.86912e+06,34.964657,-87.032349,0.0,3256,2695,2804,5184,3607,...,3187,3997,4780,4882,3973,4344,4928,3651,6758,Limestone
2,497510,3.86912e+06,34.964657,-87.027275,0.0,2983,3032,5763,2620,4831,...,3491,3357,4701,4137,3710,4007,4268,4408,2721,Limestone
3,480831,3.86889e+06,34.962391,-87.209976,0.0,3253,4592,4797,5969,3677,...,4541,3599,4573,4612,4019,5942,5880,6815,5641,Limestone
4,497278,3.86889e+06,34.962566,-87.029808,0.0,3483,3010,4030,2521,2799,...,3166,3811,5268,4506,3898,6482,4144,3887,2598,Limestone


In [17]:
@tl.curry
def tokenize(delimiter, string):
    return string.split(delimiter)
tokenizer = tokenize("_")
limestone_tokenized = list(tl.map(tokenizer, limestone_df.columns))
houston_tokenized = list(tl.map(tokenizer, houston_df.columns))
madison_tokenized = list(tl.map(tokenizer, madison_df.columns))
print("limestone")
for tk in limestone_tokenized:
    print(tk)
print("houston")
for tk in houston_tokenized:
    print(tk)
print("madison")
for tk in madison_tokenized:
    print(tk)

limestone
['X']
['Y']
['Latitude']
['Longitude']
['rf', 'irr']
['EVI', '2011', 'APR', '7', '1']
['EVI', '2011', 'APR', '23', '']
['EVI', '2011', 'MAY', '9', 'L']
['EVI', '2011', 'MAY', '25', '']
['EVI', '2011', 'JUN', '10', '']
['EVI', '2011', 'JUN', '26', '']
['EVI', '2011', 'JUL', '12', '']
['EVI', '2011', 'JUL', '28', '']
['EVI', '2011', 'AUG', '13', '']
['EVI', '2011', 'AUG', '29', '']
['EVI', '2012', 'APR', '6', 'L']
['EVI', '2012', 'APR', '22', '']
['EVI', '2012', 'MAY', '8', 'L']
['EVI', '2012', 'MAY', '24', '']
['EVI', '2012', 'JUN', '9', 'L']
['EVI', '2012', 'JUN', '25', '']
['EVI', '2012', 'JUL', '11', '']
['EVI', '2012', 'JUL', '27', '']
['EVI', '2012', 'AUG', '12', '']
['EVI', '2012', 'AUG', '28', '']
['county']
houston
['X']
['Y']
['Latitude']
['Longitude']
['rf', 'irr']
['EVI', '2011', 'Apr', '7', 'Houston', 'tif', 'rf', '1']
['EVI', '2011', 'Apr', '23', 'Houston', 'tif', 'rf']
['EVI', '2011', 'May', '9', 'Houston', 'tif', 'rf']
['EVI', '2011', 'May', '25', 'Houston', 'ti

From the tokenizers we can see that houston and limestone report date of data in columns 2, 3, 4 of tokenized output (after skipping initial rows). All dates have to hape EVI in their tokenized output. Madison uses a day of year number. Now to move all the data into a consistent format.

In [18]:
def get_date_time(tokenized_list, county):
    
    def convert_month(month):

        return {
            'jan' : "01",
            'feb' : "02",
            'mar' : "03",
            'apr' : "04",
            'may' : "05",
            'jun' : "06",
            'jul' : "07",
            'aug' : "08",
            'sep' : "09", 
            'oct' : "10",
            'nov' : "11",
            'dec' : "12"
        }[month.lower()]

    if "EVI" not in tokenized_list:
        raise ValueError("EVI not found in tokenized list. Incorrect column name being used.")

    if county != "Madison":
        return tokenized_list[1] + "-" + convert_month(tokenized_list[2]) + "-" + tokenized_list[3].zfill(2)
    else:
        year = tokenized_list[7][3:7]
        day = tokenized_list[7][7:]
        first_day = datetime.date(int(year), 1, 1)
        date = first_day + datetime.timedelta(int(day))
        return str(date.year) + "-" + str(date.month).zfill(2) + "-" + str(date.day).zfill(2)

In [19]:
def convert_cols(columns, county):
    cols = []
    for col in columns:
        tokenized = tokenize("_", col)
        if "EVI" in tokenized:
            cols.append(get_date_time(tokenized, county))
        else:
            cols.append(col)
    return cols

In [20]:
madison_df.columns = convert_cols(madison_df.columns, "Madison")
houston_df.columns = convert_cols(houston_df.columns, "Houston")
limestone_df.columns = convert_cols(limestone_df.columns, "Limestone")

In [21]:
limestone_df = limestone_df.drop(columns=["X", "Y"])
houston_df = houston_df.drop(columns=["X", "Y"])

In [22]:
print(limestone_df.columns)
print(houston_df.columns)
print(madison_df.columns)

Index(['Latitude', 'Longitude', 'rf_irr', '2011-04-07', '2011-04-23',
       '2011-05-09', '2011-05-25', '2011-06-10', '2011-06-26', '2011-07-12',
       '2011-07-28', '2011-08-13', '2011-08-29', '2012-04-06', '2012-04-22',
       '2012-05-08', '2012-05-24', '2012-06-09', '2012-06-25', '2012-07-11',
       '2012-07-27', '2012-08-12', '2012-08-28', 'county'],
      dtype='object')
Index(['Latitude', 'Longitude', 'rf_irr', '2011-04-07', '2011-04-23',
       '2011-05-09', '2011-05-25', '2011-06-10', '2011-06-26', '2011-07-12',
       '2011-07-28', '2011-08-13', '2011-08-29', '2012-04-06', '2012-04-22',
       '2012-05-08', '2012-05-24', '2012-06-09', '2012-06-25', '2012-07-11',
       '2012-07-27', '2012-08-12', '2012-08-28', 'county'],
      dtype='object')
Index(['Latitude', 'Longitude', '2011-03-23', '2011-04-08', '2011-04-24',
       '2011-05-10', '2011-05-26', '2011-06-11', '2011-06-27', '2011-07-13',
       '2011-07-29', '2011-08-14', '2011-08-30', '2012-03-22', '2012-04-07',
      

Looking at the columns of the three dataframe we can see that madison has two extra datapoint (2011-03-23 and 2011-03-22). Also the madison dates are each one day ahead of the houston and limestone days. One day makes little difference in the span of vegetation, so I will subtract one from each madison day in order to make all the column names lineup. I will also choose to drop the two extraneous days in the madison data to make all the data match up (considering there are still plenty of days for all three datasets, I do not believe this drop will have a large affect).

In [23]:
new_cols = []
for col in madison_df.columns:
    if col[0] == "2":
        tk = tokenize("-", col)
        tk[2] = str(int(tk[2])-1).zfill(2)
        new_cols.append("-".join(tk))
    else:
        new_cols.append(col)

In [24]:
madison_df.columns = new_cols
madison_df = madison_df.drop(columns=["2011-03-22", "2012-03-21"])

In [25]:
all_df = pd.concat([madison_df, houston_df, limestone_df])

C:\Users\paul.abers\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [26]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14677 entries, 0 to 6730
Data columns (total 24 columns):
2011-04-07    14677 non-null object
2011-04-23    14677 non-null object
2011-05-09    14677 non-null object
2011-05-25    14677 non-null object
2011-06-10    14677 non-null object
2011-06-26    14677 non-null object
2011-07-12    14674 non-null object
2011-07-28    14669 non-null object
2011-08-13    14677 non-null object
2011-08-29    14677 non-null object
2012-04-06    14677 non-null object
2012-04-22    14677 non-null object
2012-05-08    14677 non-null object
2012-05-24    14677 non-null object
2012-06-09    14677 non-null object
2012-06-25    14677 non-null object
2012-07-11    14657 non-null object
2012-07-27    14656 non-null object
2012-08-12    14677 non-null object
2012-08-28    14667 non-null object
Latitude      14675 non-null float64
Longitude     14675 non-null float64
county        14677 non-null object
rf_irr        14675 non-null float64
dtypes: float64(3), objec

As can be seen by the info, not all rows have data. I will drop all rows that contain a single null since it appears there are not many null values.

In [32]:
all_df_no_na = all_df.dropna()
all_df_no_na = all_df_no_na[all_df_no_na.columns[list(range(-4, len(all_df_no_na.columns)-4, 1))]]
all_df_no_na.reset_index(inplace=True)
all_df_no_na.drop(columns=["index"], inplace=True)

In [34]:
all_df_no_na.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14613 entries, 0 to 14612
Data columns (total 24 columns):
Latitude      14613 non-null float64
Longitude     14613 non-null float64
county        14613 non-null object
rf_irr        14613 non-null float64
2011-04-07    14613 non-null object
2011-04-23    14613 non-null object
2011-05-09    14613 non-null object
2011-05-25    14613 non-null object
2011-06-10    14613 non-null object
2011-06-26    14613 non-null object
2011-07-12    14613 non-null object
2011-07-28    14613 non-null object
2011-08-13    14613 non-null object
2011-08-29    14613 non-null object
2012-04-06    14613 non-null object
2012-04-22    14613 non-null object
2012-05-08    14613 non-null object
2012-05-24    14613 non-null object
2012-06-09    14613 non-null object
2012-06-25    14613 non-null object
2012-07-11    14613 non-null object
2012-07-27    14613 non-null object
2012-08-12    14613 non-null object
2012-08-28    14613 non-null object
dtypes: float64(3), obje

It appears we have not lost many values, which is good. We can now write the data out so that we can use it later in our learning stage.

In [35]:
all_df_no_na.to_csv(os.path.join(os.getcwd(), "data", "cleaned_data", "all_data.csv"))

And now we have a clean csv with all the data combined to work with.